In [212]:
import tensorflow as tf
import numpy as np
import pandas as pd
import gensim

In [265]:
train = pd.read_csv('ratings_train.txt', delimiter='\t')
test = pd.read_csv('ratings_test.txt', delimiter='\t')
y_train = train.label.values
y_test = test.label.values

In [266]:
model = gensim.models.Word2Vec.load('./vectors.bin')

In [469]:
len(model.wv.vocab)

61589

In [306]:
train_sents = np.load('train_sents.npy')
train_segs = np.load('train_segs.npy')
test_sents = np.load('test_sents.npy')
test_segs = np.load('test_segs.npy')

In [269]:
def get_max_length(seq_data):
    max_len = 0
    for sent in seq_data:
        if max_len < len(sent):
            max_len = len(sent)
    return(max_len)

In [391]:
#max_len = get_max_length(train_segs)
max_len = 10

In [392]:
word = []
for sent in train_segs:
    for seg in sent:
        word.append(seg)

In [471]:
word2num = {w:(i+1) for i,w in enumerate(set(word))}
num2word = {(i+1):w for i,w in enumerate(set(word))}

In [472]:
embedding_vector = [np.zeros(shape=300)]
for i, w in enumerate(word2num.keys()):
    if w not in model.wv.vocab:
        embedding_vector += [np.random.normal(scale=1e-2, size=300)]
    else:
        embedding_vector += [model.wv[w]]

In [473]:
word2num['<UNK>'] = 0
num2word[0] = '<UNK>'

In [474]:
def sent2index(segs):
    idx = []
    for sent in segs:
        temp = []
        for seg in sent:
            if seg not in word2num.keys():
                seg = '<UNK>'
            temp.append(word2num[seg])
        idx.append(temp)
    return idx

In [475]:
train_idx = sent2index(train_segs)
test_idx = sent2index(test_segs)

In [457]:
def get_seq_length(seq_data, max_len):
    seq_len = []
    for seq in seq_data:
        if max_len <= len(seq):
            seq_len.append(max_len)
        else:
            seq_len.append(len(seq))
    return seq_len

In [458]:
seq_length_ = get_seq_length(train_segs, max_len)

In [459]:
def give_zero_padding(seq_data, max_len):
    seq_ = []
    for seq in seq_data:
        temp = seq
        if len(temp) > max_len:
            temp = seq[:max_len]
        for _ in range(len(temp), max_len):
            temp.append(0)
        seq_.append(temp)
    return seq_

In [460]:
train_idx = give_zero_padding(train_idx, max_len)
test_idx = give_zero_padding(test_idx, max_len)

In [462]:
train_idx[0]

[4511, 37354, 27605, 21819, 32409, 60795, 7313, 31746, 0, 0]

In [436]:
learning_rate = 0.001
n_hidden = 128
n_epoch = 5
n_embedding = 300
n_step = max_len
n_input = 300
n_vocab = len(word2num)
batch_size = 64
n_output = 2

In [437]:
tf.reset_default_graph()
X = tf.placeholder(dtype=tf.int32, shape=[None, n_step])
Y = tf.placeholder(dtype=tf.int32, shape=[None])
embedding = tf.placeholder(dtype=tf.float32, shape=[n_vocab, n_embedding])
seq_len = tf.placeholder(dtype=tf.int32, shape=[None])
inputs = tf.nn.embedding_lookup(embedding, X)

In [438]:
W = tf.Variable(tf.random_normal([n_hidden, n_output]))
b = tf.Variable(tf.random_normal([n_output]))

In [439]:
cell = tf.nn.rnn_cell.GRUCell(n_hidden)
cell = tf.nn.rnn_cell.DropoutWrapper(cell, output_keep_prob=0.5)

In [440]:
outputs, states = tf.nn.dynamic_rnn(cell, inputs, sequence_length=seq_len, dtype=tf.float32)
#outputs: TensorShape([batch_size, time_steps, hidden_size])

In [441]:
idx = tf.range(tf.shape(outputs)[0])*tf.shape(outputs)[1] + (seq_len - 1)

In [442]:
last_outputs = tf.gather(tf.reshape(outputs, [-1, n_hidden]), idx)
#tf.reshape(outputs, [-1, n_hidden]): TensorShape([batch_size*time_steps, hidden_size])

In [443]:
logits = tf.matmul(last_outputs, W) + b
preds = tf.cast(tf.argmax(logits, 1), tf.int32)
correct = tf.equal(preds, Y)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [444]:
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=Y))

In [445]:
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

C:\Users\RYU\Anaconda3\envs\tensor\lib\site-packages\tensorflow\python\ops\gradients_impl.py:98: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [448]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
total_batch = int(len(train_idx) / batch_size)

for epoch in range(n_epoch):
    cost_avg = 0
    print('< epoch :', (epoch+1), '>')
    for i in range(total_batch):
        if i == (total_batch-1):
            batch_xs = train_idx[(i*batch_size):len(train_idx)]
            batch_ys = y_train[(i*batch_size):len(train_idx)]
            seq_length_data = get_seq_length(train_segs[(i*batch_size):len(train_idx)], max_len)
        else:
            batch_xs = train_idx[i*batch_size:(i+1)*batch_size]
            batch_ys = y_train[i*batch_size:(i+1)*batch_size]
            seq_length_data = get_seq_length(train_segs[(i*batch_size):(i+1)*batch_size], max_len)
        cost_val, _ = sess.run([cost, optimizer], feed_dict={X: batch_xs, Y: batch_ys, embedding:embedding_vector, seq_len: seq_length_data})
        cost_avg += cost_val
        if i % 500 == 499:
            print('%04d' % (i+1), 'Cost: ', '{:.3f}'.format(cost_avg/500))
            cost_avg = 0

< epoch : 1 >
0500 Cost:  0.758
1000 Cost:  0.714
1500 Cost:  0.701
2000 Cost:  0.691
< epoch : 2 >
0500 Cost:  0.674
1000 Cost:  0.673
1500 Cost:  0.666
2000 Cost:  0.662
< epoch : 3 >
0500 Cost:  0.654
1000 Cost:  0.653
1500 Cost:  0.652
2000 Cost:  0.648
< epoch : 4 >
0500 Cost:  0.639
1000 Cost:  0.638
1500 Cost:  0.637
2000 Cost:  0.635
< epoch : 5 >
0500 Cost:  0.626
1000 Cost:  0.627
1500 Cost:  0.625
2000 Cost:  0.621


In [449]:
test_batch = int(len(test_idx) / batch_size)
test_acc = 0

for i in range(test_batch):
    if i == (test_batch-1):
        batch_xs = test_idx[(i*batch_size):len(test_idx)]
        batch_ys = y_test[(i*batch_size):len(test_idx)]
        seq_length_data = get_seq_length(test_segs[(i*batch_size):len(test_idx)], max_len)
    else:
        batch_xs = test_idx[i*batch_size:(i+1)*batch_size]
        batch_ys = y_test[i*batch_size:(i+1)*batch_size]
        seq_length_data = get_seq_length(test_segs[(i*batch_size):(i+1)*batch_size], max_len)
    acc = sess.run(accuracy, feed_dict={X: batch_xs, Y: batch_ys, embedding:embedding_vector, seq_len: seq_length_data})
    test_acc += acc
print('Accuracy: ', '{:.3f}'.format(test_acc/test_batch))

Accuracy:  0.643
